### Normal and Reduction cell 

In [28]:
import tensorflow as tf
import tensorflow.python.keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D, SeparableConv2D, ZeroPadding2D
import tensorflow.python.keras.models
import os
from tensorflow.python.keras.layers import Lambda
import numpy as np
from tensorflow.python.keras import backend
from tensorflow.python.keras import layers, models


### Load the dataset

In [29]:

batch_size = 80
num_classes = 10
epochs = 10
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model_2.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = tensorflow.keras.utils.to_categorical(y_train, num_classes)
y_test = tensorflow.keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [30]:
def resizeInput(input1, input2, celltype):
    # resize them spatially and depth wise
    # find out less size input and apply max pooling
    # on the larger size input to make it equal size spatially
    
    if(input1.shape[0] < input2.shape[0]):
        input2 = tensorflow.keras.layers.MaxPooling2D((3, 3),
                                                        strides=(2, 2),
                                                        padding='same')(input2)
    elif input1.shape[0] > input2.shape[0]:
        input1 = tensorflow.keras.layers.MaxPooling2D((3, 3),
                                                        strides=(2, 2),
                                                        padding='same')(input1)

    # resize the input depth wise 
    # find out more size input channels and apply 1*1 conv filter on the small size input channel
    # to make both input equal depth wise
    
    print('harsh', input1.shape[2], input2.shape[2])
    
    if(input1.shape[2] < input2.shape[2]):
        input1 = tensorflow.keras.layers.Conv2D(input2.shape[2], (1, 1),
                                            strides=(1, 1),
                                            padding='same')(input1)
        input1 = tensorflow.keras.layers.Activation('relu')(input1) 
        
    elif input1.shape[2] > input2.shape[2]:
        input2 = tensorflow.keras.layers.Conv2D(input1.shape[2], (1, 1),
                                            strides=(1, 1),
                                            padding='same')(input2)
        input2 = tensorflow.keras.layers.Activation('relu')(input2) 
        
    if celltype == 'reduction':        
        input2 = tensorflow.keras.layers.Conv2D(input2.shape[2] * 2, (2, 2),
                                                strides=(1, 1),
                                                padding='same')(input2)
        input2 = tensorflow.keras.layers.Activation('relu')(input2) 
        
        input1 = tensorflow.keras.layers.Conv2D(input1.shape[2] * 2, (2, 2),
                                            strides=(1, 1),
                                            padding='same')(input1)
        input1 = tensorflow.keras.layers.Activation('relu')(input1) 

    return input1, input2
    

### Define reduction cell

In [31]:
def reduction_cell(img_input, r_img_input_2, filters, kernelSize, strides, i):    

    # changing number of filters coming from previous layers to filtersize i.e 32
    # h is input_1(0) to the cell
    # img_input_2 is input_2(1) to the cell
    # Both the input to the cell are same i.e. previous cell output.
    # I tried stacking normal and reduction cell and also implemented 
    #   skip connections but network was too slow to train. so for now keeping N = 1.    

    img_input,r_img_input_2 = resizeInput(img_input, r_img_input_2, 'reduction')
    
    h = img_input              
    
                                   
    
    # comb 2
    
    comb_2_0 = tensorflow.keras.layers.AveragePooling2D((3,3),
                                                        strides=strides,
                                                        padding="same",
                                                        name = 'r_comb_2_0_%i'%i)(h)
     
    
    comb_2_1 = tensorflow.keras.layers.SeparableConv2D(filters,
                                                       (3,3),
                                                       strides=strides,
                                                       padding='same', 
                                                       name = 'r_comb_2_1_%i'%i)(r_img_input_2)

    comb_2_1 = tensorflow.keras.layers.Activation('relu')(comb_2_1)
        
    
    comb_2_2 =  tensorflow.keras.layers.add([comb_2_0, comb_2_1])
    
    
    
    # comb 3
    
    comb_3_0 = tensorflow.keras.layers.MaxPooling2D((3, 3),
                                                    strides=(1, 1),
                                                    padding='same',  
                                                    name = 'r_comb_3_0_%i'%i)(h)

    
    comb_3_1 = tensorflow.keras.layers.MaxPooling2D((3, 3),
                                                    strides=(1, 1),
                                                    padding='same', 
                                                    name = 'r_comb_3_1_%i'%i)(r_img_input_2)
        

    comb_3_2 = tensorflow.keras.layers.add([comb_3_0, comb_3_1])
    
    
    #comb 4
    print('before', comb_2_2.shape)
    comb_4_0 = tensorflow.keras.layers.MaxPooling2D((3, 3),
                                                    strides=(1, 1),
                                                    padding='same',
                                                    name = 'r_comb_4_0_%i'%i)(h) 
    print('after', comb_4_0.shape)
        
        
    print('before', comb_2_2.shape)
    comb_4_1 = tensorflow.keras.layers.SeparableConv2D(filters,
                                                       (7,7),
                                                       strides=strides,
                                                       padding='same',
                                                       name = 'r_comb_4_1_%i'%i)(comb_2_2)
    
    comb_4_1 = tensorflow.keras.layers.Activation('relu')(comb_4_1)
    
    print('after', comb_4_1.shape)

    comb_4_2 = tensorflow.keras.layers.add([comb_4_0, comb_4_1])
    
    
     #comb 5
    
    comb_5_0 = tensorflow.keras.layers.SeparableConv2D(filters,
                                                       (7,7),
                                                       strides=strides,
                                                       padding='same',
                                                       name = 'r_comb_5_0_%i'%i)(h)
    
    comb_5_0 = tensorflow.keras.layers.Activation('relu')(comb_5_0)
       
    
  
    comb_5_1 = tensorflow.keras.layers.AveragePooling2D((3,3),
                                                        strides=strides,
                                                        padding="same",
                                                        name = 'r_comb_5_1_%i'%i)(r_img_input_2)
    

    comb_5_2 = tensorflow.keras.layers.add([comb_5_0, comb_5_1])
    
    # comb 6
         
    comb_6_0 = tensorflow.keras.layers.SeparableConv2D(filters,
                                                       (3,3),
                                                       strides=strides,
                                                       padding='same', 
                                                       name = 'r_comb_6_0_%i'%i)(comb_3_2)
    
    comb_6_0 = tensorflow.keras.layers.Activation('relu')(comb_6_0)
        
    
    
    comb_6_1 = tensorflow.keras.layers.Conv2D(filters,
                                              (1,7),
                                              strides=strides,
                                              padding='same', 
                                              name = 'r_comb_6_1_1_%i'%i)(h)
    
    comb_6_1 = tensorflow.keras.layers.Activation('relu')(comb_6_1)
        
    
    comb_6_1 = tensorflow.keras.layers.Conv2D(filters,
                                              (7,1),
                                              strides=strides,
                                              padding='same', 
                                              name = 'r_comb_6_1_2_%i'%i)(comb_6_1)
    
    comb_6_1 = tensorflow.keras.layers.Activation('relu')(comb_6_1)    
     
    
    comb_6_2 = tensorflow.keras.layers.add([comb_6_0, comb_6_1])
    
    comb_7_2 = tensorflow.keras.layers.concatenate([comb_4_2, comb_5_2, comb_6_2])    
        
    return comb_7_2, img_input

### Define normal cell

In [32]:
def normal_cell(img_input,img_input_2, filters, kernelSize, strides, i):
    
    # changing number of filters coming from previous layers to filtersize i.e 32
    # h is input_1(0) to the cell
    # img_input_2 is input_2(1) to the cell
    # Both the input to the cell are same i.e. previous cell output.
    
    img_input,img_input_2 = resizeInput(img_input, img_input_2, 'normal')
   
    h  = img_input
            
    #img_input_2  = h       
    
    
# commented out code used in stacking
                  
    
    
    # comb 2
    comb_2_0 = tensorflow.keras.layers.AveragePooling2D((3,3),
                                                        strides=strides,
                                                        name='n_comb_2_0_%i' % i,
                                                        padding="same")(h)
        

    comb_2_1 = tensorflow.keras.layers.MaxPooling2D((3, 3),
                                                    strides=(1, 1),
                                                    padding='same',
                                                    name='n_comb_2_1_%i' % i,)(h)

    comb_2_2 = tensorflow.keras.layers.add([comb_2_0, comb_2_1])

    # comb 3        
    comb_3_0 = h
    
    comb_3_1 = tensorflow.keras.layers.AveragePooling2D((3,3),
                                                        strides=(1,1),
                                                        name='n_comb_3_1_%i' % i,
                                                        padding="same")(img_input_2)

    comb_3_2 = tensorflow.keras.layers.add([comb_3_0, comb_3_1])

    # comb 4
    comb_4_1 = tensorflow.keras.layers.SeparableConv2D(filters,
                                                       (3,3),
                                                       padding = 'same',
                                                       name='n_comb_4_1_%i' % i)(img_input_2)

    comb_4_1 = tensorflow.keras.layers.Activation('relu')(comb_4_1)


    comb_4_0 = tensorflow.keras.layers.SeparableConv2D(filters,
                                                       (5, 5),
                                                       padding = 'same',
                                                       name='n_comb_4_0_%i' % i)(comb_2_2)

    comb_4_0 = tensorflow.keras.layers.Activation('relu')(comb_4_0)

    comb_4_2 =  tensorflow.keras.layers.add([comb_4_0, comb_4_1])

    # comb 5
    comb_5_0 = tensorflow.keras.layers.SeparableConv2D(filters,
                                                       (3,3),
                                                       padding = 'same',
                                                       name='n_comb_5_0_%i' % i)(comb_2_2)

    comb_5_0 = tensorflow.keras.layers.Activation('relu')(comb_5_0)

    comb_5_1 = img_input_2

    comb_5_2 = tensorflow.keras.layers.add([comb_5_0, comb_5_1])

    # comb 6
    comb_6_0 = tensorflow.keras.layers.AveragePooling2D((3,3),
                                                        strides=strides,
                                                        name='n_comb_6_0_%i' % i,
                                                        padding="same")(comb_4_2)

    comb_6_1 = tensorflow.keras.layers.SeparableConv2D(filters,
                                                       (3,3),
                                                       padding = 'same',
                                                       name='n_comb_6_1_%i' % i)(h)

    comb_6_1 = tensorflow.keras.layers.Activation('relu')(comb_6_1)

    comb_6_2 = tensorflow.keras.layers.add([comb_6_0, comb_6_1])

    comb_7_out = tensorflow.keras.layers.concatenate([comb_3_2, comb_5_2, comb_6_2])
    
    return comb_7_out, img_input

### Define Model(AmoebaNet)

In [33]:
def amoebaNet(input_tensor):
    
    i=0;
    
    h = tensorflow.keras.layers.Conv2D(filters, (3, 3),
                                       strides=(2, 2),
                                       padding='same',
                                       input_shape=(32,32,3))(input_tensor)
    
    h = tensorflow.keras.layers.Activation('relu')(h)    
       
    
    
    output_1,input_h = reduction_cell(h, h,
                                    filters = 32,
                                    kernelSize = (3,3),
                                    strides =(1,1), i=1)
        
# commented out code used in stacking

    output_2,input_h = reduction_cell(h, output_1,
                                    filters = 32,
                                    kernelSize = (3,3),
                                    strides =(1,1), i=2)

    output_3,input_h = normal_cell(output_1, output_2,
                                 filters = 32,
                                 kernelSize = (3,3) ,
                                 strides =(1,1), i=3)
    
    output_4,input_h = reduction_cell(output_2, output_3,
                                    filters = 32,
                                    kernelSize = (3,3),
                                    strides =(1,1), i=4)
    
    output_5,input_h = normal_cell(output_3, output_4,
                                 filters = 32,
                                 kernelSize = (3,3) ,
                                 strides =(1,1), i=5)
    
    output_6,input_h = reduction_cell(output_4, output_5,
                                    filters = 32,
                                    kernelSize = (3,3),
                                    strides =(1,1), i=6)
    
    output_7,input_h = normal_cell(output_5, output_6,
                                 filters = 32,
                                 kernelSize = (3,3) ,
                                 strides =(1,1), i=7)
    
        

    output = tensorflow.keras.layers.Flatten()(output_7)
    
    softmax_output = tensorflow.keras.layers.Dense(10, activation='softmax')(output)        
    
    model = models.Model(input_tensor, softmax_output, name='ameoba net')        
    
    return model

In [34]:
def resizeInput(input1, input2, celltype):
    # resize them spatially and depth wise
    # find out less size input and apply max pooling
    # on the larger size input to make it equal size spatially  
    #print(input1.shape, input2.shape, celltype)
    
    if(input1.shape[1] < input2.shape[1]):
        print(0)
        input2 = tensorflow.keras.layers.MaxPooling2D((3, 3),
                                                        strides=(2, 2),
                                                        padding='same')(input2)
    elif input1.shape[1] > input2.shape[1]:
        print(0)
        input1 = tensorflow.keras.layers.MaxPooling2D((3, 3),
                                                        strides=(2, 2),
                                                        padding='same')(input1)
    
    # resize the input depth wise 
    # find out more size input channels and apply 1*1 conv filter on the small size input channel
    # to make both input equal depth wise
    
    #print(input1.shape, input2.shape, celltype)
    
    if(input1.shape[3] < input2.shape[3]):
        print(1)
        input1 = tensorflow.keras.layers.Conv2D(input2.shape[3], (1, 1),
                                            strides=(1, 1),
                                            padding='same')(input1)
        input1 = tensorflow.keras.layers.Activation('relu')(input1) 
        
    elif input1.shape[3] > input2.shape[3]:        
        input2 = tensorflow.keras.layers.Conv2D(input1.shape[3], (1, 1),
                                            strides=(1, 1),
                                            padding='same')(input2)
        input2 = tensorflow.keras.layers.Activation('relu')(input2) 
        
    if celltype == 'reduction':        
        input2 = tensorflow.keras.layers.Conv2D(input2.shape[3], (1, 1),
                                                strides=(1, 1),
                                                padding='same')(input2)
        input2 = tensorflow.keras.layers.Activation('relu')(input2) 
        
        input1 = tensorflow.keras.layers.Conv2D(input1.shape[3], (1, 1),
                                            strides=(1, 1),
                                            padding='same')(input1)
        input1 = tensorflow.keras.layers.Activation('relu')(input1) 

    return input1, input2
    

In [36]:
filters =32
x = x_train[:batch_size]
x=tf.to_float(x, name='ToFloat')
img_input = tensorflow.keras.layers.Input(shape = (32,32,3), batch_size =batch_size, dtype='int32')
print(img_input.shape)
model = amoebaNet(img_input)

(80, 32, 32, 3)


TypeError: Failed to convert object of type <class 'tuple'> to Tensor. Contents: (1, 1, 32, Dimension(32)). Consider casting elements to a supported type.

### Network Summary

In [21]:
print(model.summary())

Model: "ameoba net"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(80, 32, 32, 3)]    0                                            
__________________________________________________________________________________________________
conv2d_22 (Conv2D)              (80, 16, 16, 32)     896         input_5[0][0]                    
__________________________________________________________________________________________________
activation_132 (Activation)     (80, 16, 16, 32)     0           conv2d_22[0][0]                  
__________________________________________________________________________________________________
up_sampling2d_114 (UpSampling2D (80, 32, 32, 32)     0           activation_132[0][0]             
_________________________________________________________________________________________

### Optimizer and loss

In [95]:
opt = tensorflow.keras.optimizers.RMSprop(lr=0.0003, decay=1e-6, )

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

### Preprocessing the data

In [96]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
y_train = y_train

### Train model

In [97]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          validation_data=(x_test, y_test),
          epochs=epochs)

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 212s 4ms/sample - loss: 1.5370 - acc: 0.4564 - val_loss: 1.3619 - val_acc: 0.5199
Epoch 2/10
50000/50000 [==============================] - 212s 4ms/sample - loss: 1.2042 - acc: 0.5778 - val_loss: 1.1030 - val_acc: 0.6042
Epoch 3/10
50000/50000 [==============================] - 211s 4ms/sample - loss: 1.0596 - acc: 0.6281 - val_loss: 1.0016 - val_acc: 0.6449
Epoch 4/10
50000/50000 [==============================] - 211s 4ms/sample - loss: 0.9614 - acc: 0.6651 - val_loss: 0.9879 - val_acc: 0.6521
Epoch 5/10
50000/50000 [==============================] - 212s 4ms/sample - loss: 0.8895 - acc: 0.6909 - val_loss: 0.9644 - val_acc: 0.6612
Epoch 6/10
50000/50000 [==============================] - 212s 4ms/sample - loss: 0.8315 - acc: 0.7132 - val_loss: 0.9265 - val_acc: 0.6752
Epoch 7/10
50000/50000 [==============================] - 212s 4ms/sample - loss: 0.7790 - acc: 0.7310 - val_l

### Save Model

In [98]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)



Saved trained model at /home/vikas/Desktop/NewFolder/saved_models/keras_cifar10_trained_model_2.h5 


### Evaluate

In [99]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 10s 1ms/sample - loss: 0.9151 - acc: 0.6856
Test loss: 0.9150700416564942
Test accuracy: 0.6856
